# Basic I/O of DFPT
The density functional perturbation theory (DFPT) specific methods and attributes are available in 'crystal_io.py' since July 2022.

## Tests

4 tests are performed. The first one is for $\Gamma$ point frequencies. The second one is for dispersion calculations. The third one has negative frequencies, in order to examine the input option `rm_imaginary`. The last one is to illustrate the general applicability of this module to QHA outputs.

1. Form I paracetamol, 'freqf1-r0.out', $\Gamma$ point HA frequencies, with occasional warning messages. nqpoint = 1, nmode = 240, natom = 80.  
2. A graphene primitive cell, 'nostr-modes.out', phonon dispersion calculated along the $\Gamma-K-M$ path. nqpoint = 123, nmode = 6, natom = 2.  
3. Form II paracetamol, 'f2p2q-r1.out', $\Gamma$ point HA frequencies, with imaginary modes. nqpoint = 1, nmode = 480, natom = 160.  
4. Corundum, 'corundum.out', available on [crystal tutorial website](https://tutorials.crystalsolutions.eu/tutorials/Tutorial_QHA/AL2O3_SC30_QHA.out). Used to illustrate the structure of data when reading outputs by the QHA module of CRYSTAL. 

### Get energy, qpoints and frequencies at each qpoint

nqpoint = 1, nmode = 240 for paracetamol case, nqpoint = 120, nmode = 6 for graphene case. 123 q points exist in the graphene case, with 3 of them overlapping with the previous run, which, by default, is removed. To keep them, set `rm_overlap` = False.


In both cases, nqpoint should be an integer and nmode should be a numpy array.

In [1]:
from CRYSTALpytools.crystal_io import Crystal_output

paracetamol = Crystal_output()
paracetamol.read_cry_output('data/freqf1-r0.out')
paracetamol.get_phonon()
print('Paracetamol EDFT:', paracetamol.edft[0])
print('Paracetamol qpoint:', paracetamol.nqpoint, paracetamol.qpoint)
print('Paracetamol modes:', len(paracetamol.nmode), paracetamol.nmode)

Paracetamol EDFT: -5402456.254965135
Paracetamol qpoint: 1 [[array([0., 0., 0.]), 1.0]]
Paracetamol modes: 1 [240]


In [2]:
graphene = Crystal_output()
graphene.read_cry_output('data/nostr_modes.out')
graphene.get_phonon()
print('Graphene EDFT:', graphene.edft[0])
print('Graphene qpoints:', graphene.nqpoint)
print('Graphene modes:', len(graphene.nmode), graphene.nmode)

Graphene EDFT: -28768034.906402458
Graphene qpoints: 120
Graphene modes: 120 [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6]


/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:1007: UserWarning: Negative frequencies detected.
  Calculated thermodynamics might be inaccurate. Negative frequencies will be substituted by NaN.
  self = PhononBASE.clean_imaginary(self, threshold=imaginary_tol)
/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:1010: UserWarning: Overlap of q points is detected between q points   0 and 122
  self = PhononBASE.clean_q_overlap(self, threshold=q_overlap_tol)
/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:1010: UserWarning: Overlap of q points is detected between q points  40 and  41
  self = PhononBASE.clean_q_overlap(self, threshold=q_overlap_tol)
/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:1010: UserWarning: Overlap of q points is d

### Get eigenvectors
Self.eigenvector should be a 1 (nqpoint) * 240 (nmode) * 80 (natom) * 3 (xyz) complex array for paracetamol case. A 123 * 6 * 2 * 3 complex array for graphene case. Overlapping q points and imaginary modes are kept.

Example: Find the qpoint coordinate, frequency and eigenvectors of all atoms in the graphene primitive cell, with a given qpoint (75) and vibrational mode (3rd)

In [3]:
paracetamol.get_phonon(read_eigvt=True, rm_imaginary=False, rm_overlap=False)

print('Paracetamol')
print('nqpoint=', len(paracetamol.eigenvector),
      'nmode=', len(paracetamol.eigenvector[0]),
      'natom=', len(paracetamol.eigenvector[0, 0]),
      'first eigenvector=', paracetamol.eigenvector[0, 0, 0])

graphene.get_phonon(read_eigvt=True, rm_imaginary=False, rm_overlap=False)
print('Graphene')
print('nqpoint=', len(graphene.eigenvector),
      'nmode=', len(graphene.eigenvector[0]),
      'natom=', len(graphene.eigenvector[0, 0]),
      'first eigenvector=', graphene.eigenvector[0, 0, 0])

Paracetamol
nqpoint= 1 nmode= 240 natom= 80 first eigenvector= [ 0.11126206+0.j  0.        +0.j -0.01098885+0.j]
Graphene
nqpoint= 123 nmode= 6 natom= 2 first eigenvector= [0.        +0.j 0.        +0.j 0.70710678+0.j]


In [4]:
print('Wavevector coordinates:', graphene.qpoint[74][0])
print('Frequency:', graphene.frequency[74, 2])
print('Eigenvectors:', graphene.eigenvector[74, 2])

Wavevector coordinates: [0.47083333 0.05833333 0.        ]
Frequency: 19.8656
Eigenvectors: [[ 3.46485288e-04+0.02078912j  7.06829987e-01+0.j
  -3.46485288e-04+0.j        ]
 [-3.46485288e-03-0.02044263j  6.95049487e-01-0.12819956j
   3.46485288e-04+0.j        ]]


Compare with the crystal output file:

At the start point of $K-M$ path:  
```
 *******************************************************************************

  PHONONS ALONG PATH:   2 NUMBER OF K POINTS:   41

  FROM K  (   2   2   0 ) TO K  (   3   0   0 ) WITH DENOMINATOR    6

  THE POSITION OF THE POINTS IS EXPRESSED IN UNITS        OF DENOMINATOR  240

 *******************************************************************************
```
 
 At the 75th qpoint:  
```
 DISPERSION K POINT NUMBER    75 COORD:  C( 113  14   0 )    WEIGHT:    1.

    MODES         EIGV          FREQUENCIES     IRREP
             (HARTREE**2)   (CM**-1)     (THZ)
    1-   1    0.4928E-05    487.1942   14.6057  (  1)
    2-   2    0.8383E-05    635.4696   19.0509  (  1)
    3-   3    0.9116E-05    662.6443   19.8656  (  1)
    4-   4    0.3933E-04   1376.3503   41.2619  (  1)
    5-   5    0.4131E-04   1410.6170   42.2892  (  1)
    6-   6    0.4439E-04   1462.3150   43.8391  (  1)

 MODES IN PHASE

 FREQ(CM**-1)    487.19    635.47    662.64   1376.35   1410.62   1462.32

 AT.   1 C  X     0.0001    0.0001    0.0001   -0.2006    0.1997    0.0072
            Y     0.0000   -0.0001    0.2040    0.0015   -0.0002   -0.1963
            Z     0.2041   -0.2007   -0.0001    0.0001    0.0001    0.0000
 AT.   2 C  X    -0.0001    0.0001   -0.0010    0.2041    0.1963    0.0003
            Y     0.0000   -0.0001    0.2006   -0.0004   -0.0077    0.1996
            Z     0.2007    0.2041    0.0001    0.0001   -0.0001   -0.0000

 MODES IN ANTI-PHASE

 FREQ(CM**-1)    487.19    635.47    662.64   1376.35   1410.62   1462.32

 AT.   1 C  X    -0.0000    0.0000    0.0060   -0.0368    0.0002   -0.0420
            Y     0.0000   -0.0000   -0.0000   -0.0058    0.0425   -0.0362
            Z    -0.0000   -0.0372    0.0000    0.0000    0.0000   -0.0000
 AT.   2 C  X     0.0000    0.0000   -0.0059   -0.0001   -0.0361   -0.0425
            Y     0.0000   -0.0000   -0.0370   -0.0060   -0.0418    0.0000
            Z    -0.0372   -0.0000   -0.0000    0.0000    0.0000    0.0000
```


### Get rid of negative frequency
A test is performed on a harmonic phonon output with negative frequencies. In principle, the user is expected to redo all calculations to get rid of imaginary modes, instead of continuing thermodynamic analysis. Here we assume the user does not know the existance of imagninary modes.

Removing negative frequency is launched by default with `rm_imaginary` = True.

Tests performed on Form II paracetamol $\Gamma$ point, stretched by 2% volumetric strain. There are 160 atoms, 480 modes, 1 qpoint. The output is attached in the same directory named as 'f2p2q-r1.out'. Modes 1-5 are negative, as given below:

```
    MODES         EIGV          FREQUENCIES     IRREP  IR   INTENS    RAMAN
             (HARTREE**2)   (CM**-1)     (THZ)             (KM/MOL)
    1-   1   -0.7201E-07    -58.8941   -1.7656  (B3g)   I (     0.00)   A
    2-   2   -0.6031E-07    -53.9001   -1.6159  (Ag )   I (     0.00)   A
    3-   3   -0.3525E-07    -41.2052   -1.2353  (Au )   I (     0.00)   I
    4-   4   -0.1932E-07    -30.5064   -0.9146  (B2g)   I (     0.00)   A
    5-   5   -0.2848E-08    -11.7124   -0.3511  (B1g)   I (     0.00)   A
    6-   6   -0.4132E-19      0.0000    0.0000  (B2u)   A (     0.00)   I
```

PS: Never use pob-TZVP basis set for energies, that is miserable :-)

In [5]:
neg_freq = Crystal_output()
neg_freq.read_cry_output('data/f2p2q-r1.out')
neg_freq.get_phonon(read_eigvt=True)

print(neg_freq.frequency[0, :6])
print(neg_freq.eigenvector[0, :6, 0, :])

[nan nan nan nan nan  0.]
[[        nan+0.j         nan+0.j         nan+0.j]
 [        nan+0.j         nan+0.j         nan+0.j]
 [        nan+0.j         nan+0.j         nan+0.j]
 [        nan+0.j         nan+0.j         nan+0.j]
 [        nan+0.j         nan+0.j         nan+0.j]
 [-0.07905694+0.j  0.        +0.j  0.        +0.j]]


### Read outputs from QHA module

The QHA module of CRYSTAL includes automatic generation, optimization and frequency calculations at $\Gamma$ point. Therefore a different strategy is adopted and keeps the structure of code consistent. Since in all cases, nqpoint = 1, this dimension is removed to avoid an extra 'ncalc' dimension. `Crystal_output().frequency` becomes a 'ncalc\*nmode' list that corresponds to frequencies at each volume, or HA phonon calculation, instead of at each q point. Other attributes are changed accordingly.

**NOTE**

When this method is called, `rm_overlap` MUST be false. Reading it with `CRYSTALpytools.thermodynamics.Quasi_harmonic.from_QHA_file` method is recommended.

In [6]:
qha_freq = Crystal_output()
qha_freq.read_cry_output('data/corundum.out')
qha_freq.get_phonon(read_eigvt=True, rm_overlap=False)

print(qha_freq.nqpoint)
print(qha_freq.qpoint)
print(qha_freq.nmode)

4
[[array([0., 0., 0.]), 1.0], [array([0., 0., 0.]), 1.0], [array([0., 0., 0.]), 1.0], [array([0., 0., 0.]), 1.0]]
[90 90 90 90]
